# Data exploration for city of Bremen

## Import required packages

In [18]:
import pandas as pd
import os
import folium
import geopandas as gpd
from shapely.ops import cascaded_union

## Import raw data

In [4]:
raw_data = pd.read_csv('../data/raw/bremen.csv')
raw_data.drop(columns=['Unnamed: 0'], inplace=True) # Drop second index column
raw_data.head()

,p_spot,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,b_bike_type,p_name,p_number,p_lng,p_bike
0,True,0,2019-01-20 02:06:00,11281,first,4774295,5,50.808852,15,Biegenstraße/Cineplex,5155.0,8.773134,False
1,True,0,2019-01-20 14:16:00,11281,last,4774295,4,50.808852,15,Biegenstraße/Cineplex,5155.0,8.773134,False
2,True,0,2019-01-20 00:00:00,11169,first,4774543,5,50.795224,15,Südbahnhof,5173.0,8.763266,False
3,True,0,2019-01-20 01:55:00,11169,start,4774543,5,50.795224,15,Südbahnhof,5173.0,8.763266,False
4,True,0,2019-01-20 02:06:00,11169,end,4774368,4,50.804522,15,Frankfurter Straße/Psychologie,5159.0,8.770358,False


## Filter data for entrys of Bremen only

### Import geodata of Bremen (PLZ based)

https://public.opendatasoft.com/explore/dataset/postleitzahlen-deutschland/table/?refine.note=Bremen

In [14]:
plz_df = gpd.read_file('../data/external/plz_bremen.geojson')

In [10]:
# Function that creates map with center such that all of Bremen is displayed.

def create_bremen_map(zoom = 11):
    
    m = folium.Map(location=[53.122962,8.7515937], zoom_start=zoom)
    
    return m

In [15]:
geo_data_bremen = create_bremen_map()

folium.Choropleth(plz_df).add_to(geo_data_bremen)

geo_data_bremen

### Get "boundaries" (min, max of latitude and longitude) of Bremen

In [25]:
boundaries = cascaded_union(plz_df.geometry).bounds
boundaries

(8.4815929, 53.0110367, 8.9907318, 53.22892479999999)

### Filter raw data such that we only keep data points within the boundaries of Bremen

In [29]:
bremen = raw_data[(raw_data['p_lat'] < boundaries[3]) &
         (raw_data['p_lat'] > boundaries[1]) &
         (raw_data['p_lng'] < boundaries[2])  &
         (raw_data['p_lng'] > boundaries[0])]

print(len(bremen))

1309128


In [32]:
bremen.sample(3)

,p_spot,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,b_bike_type,p_name,p_number,p_lng,p_bike
3540081,False,12,2019-12-04 15:36:00,20644,start,25628699,1,53.084316,71,BIKE 20644,0.0,8.815424,True
2979365,True,0,2019-02-24 14:30:00,20890,end,7873840,5,53.106418,71,Universum Bremen,2912.0,8.845743,False
2879371,False,12,2019-12-11 21:41:00,20712,end,25896302,1,53.089258,71,BIKE 20712,0.0,8.804102,True


## Reorder data frame

In [33]:
bremen = bremen[['datetime', 'b_number', 'p_spot', 'p_place_type',
                 'trip', 'p_uid', 'p_bikes', 'b_bike_type', 'p_name',
                 'p_number', 'p_bike', 'p_lat', 'p_lng']]

In [34]:
bremen.sample(3)

,datetime,b_number,p_spot,p_place_type,trip,p_uid,p_bikes,b_bike_type,p_name,p_number,p_bike,p_lat,p_lng
3454853,2019-10-22 23:59:00,20672,True,0,last,7873706,2,71,Pusdorfer Marktplatz,2959.0,False,53.081250,8.772694
2496485,2019-05-25 19:55:00,20799,False,12,end,16269467,1,71,BIKE 20799,0.0,True,53.069529,8.824298
3338634,2019-09-01 08:48:00,20755,True,0,end,15767205,1,71,Bahnhof Walle,2918.0,False,53.103159,8.786058


# First explorations

## Total number of bookings (data points)

In [35]:
len(bremen)

1309128

## Total number of null values

- there are only null values (88) of the attribute p_number

In [36]:
bremen.isna().sum()

datetime         0
b_number         0
p_spot           0
p_place_type     0
trip             0
p_uid            0
p_bikes          0
b_bike_type      0
p_name           0
p_number        88
p_bike           0
p_lat            0
p_lng            0
dtype: int64

## Total number of bikes - 443
- there are 443 different bikes presented in our data
- WESER-KURIER announced to provide 450 https://www.nextbike.de/de/news/start-fuer-wk-bike-in-bremen/

In [37]:
print(len(bremen.b_number.unique()))
print(bremen.b_number.unique())

443
[20668 20649 20964 20866 20799 20676 20855 20707 20815 20891 20721 20894
 20958 20942 20635 20838 20889 20827 20793 20797 20748 20791 20713 20790
 20810 20881 20817 20614 20768 20693 20626 20946 20915 20788 20684 20902
 20816 20704 20736 20912 20907 20726 20769 20876 20943 20872 20924 20716
 20968 20761 20686 20804 20703 20967 20916 20756 20744 20709 20725 20918
 20903 20702 20930 20688 20819 20690 20938 20843 20727 20874 20849 20651
 20742 20751 20658 20648 20733 20904 20846 20697 20873 20929 20669 20789
 20859 20757 20928 20854 20821 20835 20800 20771 20833 20937 20839 20935
 20794 20715 20893 20621 20747 20900 20667 20957 20712 20625 20657 20856
 20888 20617 20766 20864 20724 20645 20851 20805 20939 20921 20844 20731
 20722 20934 20629 20837 20719 20842 20899 20734 20698 20616 20682 20743
 20802 20735 20917 20858 20887 20798 20673 20678 20932 20706 20896 20677
 20806 20730 20732 20773 20630 20739 20679 20687 20784 20647 20822 20832
 20825 20738 20650 20685 20674 20962 20906 2080

### For a specific bike (b_number = 20668) check trips
- there seem to be many duplicates

In [19]:
bremen[bremen.b_number == 20668].sort_values('datetime')[:20]

,p_spot,p_place_type,datetime,b_number,trip,p_uid,p_bikes,p_lat,b_bike_type,p_name,p_number,p_lng,p_bike
0,False,12,2019-01-20 00:00:00,20668,first,12097754,1,53.081698,71,BIKE 20668,0.0,8.812411,True
654495,False,12,2019-01-20 00:00:00,20668,first,12097754,1,53.081698,71,BIKE 20668,0.0,8.812411,True
1,False,12,2019-01-20 01:33:00,20668,last,12097754,1,53.081767,71,BIKE 20668,0.0,8.812408,True
654496,False,12,2019-01-20 01:33:00,20668,last,12097754,1,53.081767,71,BIKE 20668,0.0,8.812408,True
1583,True,0,2019-01-22 09:15:00,20668,first,7873316,5,53.083167,71,Hauptbahnhof / Übersee Museum,2946.0,8.811472,False
656078,True,0,2019-01-22 09:15:00,20668,first,7873316,5,53.083167,71,Hauptbahnhof / Übersee Museum,2946.0,8.811472,False
1584,True,0,2019-01-22 09:42:00,20668,start,7873316,5,53.083167,71,Hauptbahnhof / Übersee Museum,2946.0,8.811472,False
656079,True,0,2019-01-22 09:42:00,20668,start,7873316,5,53.083167,71,Hauptbahnhof / Übersee Museum,2946.0,8.811472,False
1585,False,12,2019-01-22 09:49:00,20668,end,12155136,1,53.080356,71,BIKE 20668,0.0,8.795718,True
656080,False,12,2019-01-22 09:49:00,20668,end,12155136,1,53.080356,71,BIKE 20668,0.0,8.795718,True


## Check for duplicates

In [41]:
# Checking for duplicates regarding the timestamp and bike_number 
bremen.duplicated(subset=['datetime', 'b_number']).sum()

654564

## Drop duplicates - half of data (there is perfectly one duplicate per data point)

In [43]:
bremen = bremen[bremen.duplicated(subset=['datetime', 'b_number'], keep='first') == False]

In [44]:
len(bremen)

654564

In [45]:
bremen.isna().sum()

datetime         0
b_number         0
p_spot           0
p_place_type     0
trip             0
p_uid            0
p_bikes          0
b_bike_type      0
p_name           0
p_number        44
p_bike           0
p_lat            0
p_lng            0
dtype: int64

## Specifications of trip
- [start, end, first, last]  (do not refer to duplicates)

In [46]:
bremen.trip.unique()

array(['first', 'last', 'start', 'end'], dtype=object)

In [47]:
len(bremen[(bremen.trip != 'first') & (bremen.trip != 'last')])

462469

In [51]:
bremen['datetime'] = pd.to_datetime(bremen['datetime']) # parse timestamp to datetime

### Sort data frame by timestamp

In [53]:
bremen = bremen.sort_values('datetime')

In [54]:
bremen[bremen.b_number == 20668][:20]

,datetime,b_number,p_spot,p_place_type,trip,p_uid,p_bikes,b_bike_type,p_name,p_number,p_bike,p_lat,p_lng
2241283,2019-01-20 00:00:00,20668,False,12,first,12097754,1,71,BIKE 20668,0.0,True,53.081698,8.812411
2241284,2019-01-20 01:33:00,20668,False,12,last,12097754,1,71,BIKE 20668,0.0,True,53.081767,8.812408
2242946,2019-01-22 09:15:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472
2242947,2019-01-22 09:42:00,20668,True,0,start,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472
2242948,2019-01-22 09:49:00,20668,False,12,end,12155136,1,71,BIKE 20668,0.0,True,53.080356,8.795718
2242949,2019-01-22 14:52:00,20668,False,12,start,12155136,1,71,BIKE 20668,0.0,True,53.080453,8.795549
2242950,2019-01-22 15:35:00,20668,False,12,end,12162204,1,71,BIKE 20668,0.0,True,53.062169,8.875182
2242951,2019-01-22 23:59:00,20668,False,12,last,12162204,1,71,BIKE 20668,0.0,True,53.061929,8.875118
2243889,2019-01-23 00:00:00,20668,False,12,first,12162204,1,71,BIKE 20668,0.0,True,53.061929,8.875118
2243890,2019-01-23 10:42:00,20668,False,12,last,12162204,1,71,BIKE 20668,0.0,True,53.062032,8.875321


In [55]:
bremen[(bremen.b_number == 20668) & ((bremen.trip == 'first') & (bremen.datetime.dt.hour != 0))]

,datetime,b_number,p_spot,p_place_type,trip,p_uid,p_bikes,b_bike_type,p_name,p_number,p_bike,p_lat,p_lng
2242946,2019-01-22 09:15:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472
2246583,2019-01-25 07:46:00,20668,True,0,first,7873007,4,71,Domsheide,2933.0,False,53.074592,8.810182
2264984,2019-02-11 13:24:00,20668,True,0,first,7873893,2,71,Woltmershauser Str.,2966.0,False,53.078528,8.779167
2311718,2019-02-28 14:19:00,20668,False,12,first,13051288,1,71,BIKE 20668,0.0,True,53.081902,8.810073
2313106,2019-03-01 16:45:00,20668,False,12,first,13075883,1,71,BIKE 20668,0.0,True,53.070702,8.830958
2348561,2019-03-19 09:28:00,20668,False,12,first,13428348,1,71,BIKE 20668,0.0,True,53.078944,8.848838
2356371,2019-03-25 10:56:00,20668,True,0,first,8638942,4,71,Föhrenstraße,2972.0,False,53.064904,8.879091
2369451,2019-04-01 06:33:00,20668,False,12,first,13917223,1,71,Hamburger Straße 248,0.0,True,53.068182,8.849903
2441464,2019-04-30 08:29:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472
2445210,2019-05-01 03:51:00,20668,True,0,first,7873316,5,71,Hauptbahnhof / Übersee Museum,2946.0,False,53.083167,8.811472
